In [8]:
from dotenv import load_dotenv
import os

# 加载 .env 文件
load_dotenv()

print("✅ API Keys 已从 .env 加载")
print(f"OpenAI Key: {os.getenv('OPENAI_API_KEY')[:20]}...")  # 只显示前20个字符

✅ API Keys 已从 .env 加载
OpenAI Key: sk-proj-jkQQMRGnfedB...


In [12]:
# -*- coding: utf-8 -*-
"""
Pilot Study - Step 1: Minimal session collection (20–30 sessions)
Models: GPT-5 (OpenAI), Claude (Anthropic), DeepSeek, Baidu ERNIE (Qianfan)

Usage:
  python pilot_collect_sessions_gpt5_claude_deepseek_baidu.py \
      --csv debate_topic.csv \
      --outdir sessions \
      --n_draws 12 \
      --min_domains 4 \
      --seed 42 \
      --debug 1

Notes:
- Set API keys via environment variables:
  OPENAI_API_KEY, ANTHROPIC_API_KEY, DEEPSEEK_API_KEY, BAIDU_QIANFAN_AK, BAIDU_QIANFAN_SK
- Set DRY_RUN=1 to skip real API calls (placeholders will be written).
"""

import os
import re
import csv
import json
import time
import random
import argparse
from dataclasses import dataclass
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import requests
import pandas as pd


# ==================== Global Configuration ====================

DRY_RUN = os.getenv("DRY_RUN", "0") == "1"

def env(k: str) -> str:
    return os.getenv(k, "")

API_KEYS = {
    "openai": env("OPENAI_API_KEY"),
    "anthropic": env("ANTHROPIC_API_KEY"),
    "deepseek": env("DEEPSEEK_API_KEY"),
    "baidu_ak": env("BAIDU_QIANFAN_AK"),
    "baidu_sk": env("BAIDU_QIANFAN_SK"),
}

# Debate rounds (与你原设计一致：4个回合 × 2方 = 8发言)
DEBATE_ROUNDS = [
    ("Pro First Speaker", "Round 1: Pro Opening Statement"),
    ("Con First Speaker", "Round 1: Con Opening Statement"),
    ("Pro Second Speaker", "Round 2: Pro Rebuttal/Supplement"),
    ("Con Second Speaker", "Round 2: Con Rebuttal/Supplement"),
    ("Pro First Speaker", "Round 3: Pro Cross-examination"),
    ("Con First Speaker", "Round 3: Con Cross-examination"),
    ("Pro Second Speaker", "Round 4: Pro Closing Statement"),
    ("Con Second Speaker", "Round 4: Con Closing Statement"),
]


# ==================== Utilities ====================

def slugify(text: str, maxlen: int = 40) -> str:
    t = re.sub(r"\s+", "_", str(text).strip())
    t = re.sub(r"[^a-zA-Z0-9_\-]+", "", t)
    return t[:maxlen]

def ensure_outdir(path: str):
    os.makedirs(path, exist_ok=True)

def choose_domain_column(df: pd.DataFrame) -> Optional[str]:
    for c in ["Domain", "domain", "PolicyDomain", "Category", "类别", "领域"]:
        if c in df.columns:
            return c
    return None

def make_order_vector(n_draws: int, seed: int) -> List[str]:
    rng = random.Random(seed)
    n_zh = n_draws // 2
    n_en = n_draws - n_zh
    arr = ["zh-first"] * n_zh + ["en-first"] * n_en
    rng.shuffle(arr)
    return arr

def stratified_sample(
    df: pd.DataFrame,
    domain_col: Optional[str],
    n_draws: int,
    min_domains: int,
    seed: int
) -> pd.DataFrame:
    rng = random.Random(seed)
    df = df.copy()
    df = df[df["Motion"].astype(str).str.len() > 0]
    df = df[df["Motion_Chinese"].astype(str).str.len() > 0]

    if domain_col and domain_col in df.columns:
        uniq = list(df[domain_col].dropna().astype(str).unique())
        if not uniq:
            return df.sample(n=min(n_draws, len(df)), random_state=seed).reset_index(drop=True)
        K = min(max(min_domains, 3), min(5, len(uniq)))
        rng.shuffle(uniq)
        chosen = uniq[:K]
        base = n_draws // K
        rem = n_draws - base * K
        out = []
        for i, d in enumerate(chosen):
            block = df[df[domain_col].astype(str) == d]
            take = min(len(block), base + (1 if i < rem else 0))
            if take > 0:
                out.append(block.sample(n=take, random_state=seed + i))
        sampled = pd.concat(out, axis=0) if out else pd.DataFrame(columns=df.columns)
        if len(sampled) < n_draws:
            rest = df.loc[~df.index.isin(sampled.index)]
            need = n_draws - len(sampled)
            if need > 0 and len(rest) > 0:
                sampled = pd.concat([sampled, rest.sample(n=min(need, len(rest)), random_state=seed+999)], axis=0)
        return sampled.sample(frac=1.0, random_state=seed).reset_index(drop=True)
    else:
        print("[WARN] No domain column; using simple random sample.")
        return df.sample(n=min(n_draws, len(df)), random_state=seed).reset_index(drop=True)


# ==================== Providers (HTTP) ====================

def call_openai_gpt5(prompt: str) -> str:
    if DRY_RUN:
        return f"[DRY_RUN GPT-5] {prompt[:140]} ..."
    if not API_KEYS["openai"]:
        return "[OpenAI API key missing]"
    url = "https://api.openai.com/v1/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEYS['openai']}", "Content-Type": "application/json"}
    data = {
        "model": "gpt-5",  # 你的目标模型名
        "messages": [{"role": "user", "content": prompt}],
    }
    r = requests.post(url, headers=headers, json=data, timeout=60)
    if r.status_code == 200:
        js = r.json()
        return js["choices"][0]["message"]["content"]
    return f"[OpenAI error {r.status_code}] {r.text[:200]}"

def call_anthropic_claude(prompt: str, model: str = "claude-sonnet-4-5-20250929") -> str:
    if DRY_RUN:
        return f"[DRY_RUN Claude] {prompt[:140]} ..."
    if not API_KEYS["anthropic"]:
        return "[Anthropic API key missing]"
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "x-api-key": API_KEYS["anthropic"],
        "Content-Type": "application/json",
        "anthropic-version": "2025-09-29",
    }
    data = {"model": model,  "messages": [{"role": "user", "content": prompt}]}
    r = requests.post(url, headers=headers, json=data, timeout=60)
    if r.status_code == 200:
        js = r.json()
        return js["content"][0]["text"]
    return f"[Anthropic error {r.status_code}] {r.text[:200]}"

def call_deepseek(prompt: str, model: str = "deepseek-chat") -> str:
    if DRY_RUN:
        return f"[DRY_RUN DeepSeek] {prompt[:140]} ..."
    if not API_KEYS["deepseek"]:
        return "[DeepSeek API key missing]"
    url = "https://api.deepseek.com/v1/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEYS['deepseek']}", "Content-Type": "application/json"}
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
    }
    r = requests.post(url, headers=headers, json=data, timeout=60)
    if r.status_code == 200:
        js = r.json()
        return js["choices"][0]["message"]["content"]
    return f"[DeepSeek error {r.status_code}] {r.text[:200]}"

# ----- Baidu Qianfan (AK/SK -> access_token -> chat) -----

def call_baidu_ernie(prompt: str, model: str = "ernie-4.5-turbo-128k") -> str:
    """
    使用已直接持有的 Baidu Qianfan access_token (形如 bce-v3/ALTAK-...)。
    无需 AK/SK。
    """
    if DRY_RUN:
        return f"[DRY_RUN Baidu] {prompt[:140]} ..."
    token = API_KEYS.get("baidu_ak")  # 这里直接从环境变量里读你的 access_token
    if not token:
        return "[Baidu access_token missing]"
    url = "https://qianfan.baidubce.com/v2/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"  # 直接带 bce-v3/... token
    }
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
    }
    try:
        r = requests.post(url, headers=headers, json=payload, timeout=60)
        if r.status_code == 200:
            js = r.json()
            if "result" in js:
                return js["result"]
            if "choices" in js and js["choices"]:
                return js["choices"][0].get("message", {}).get("content", "")
            return json.dumps(js)[:200]
        return f"[Baidu error {r.status_code}] {r.text[:200]}"
    except Exception as e:
        return f"[Baidu exception] {e}"


# ==================== Debate Core (one session) ====================

ROLES_ORDER = [
    "Pro First Speaker",
    "Con First Speaker",
    "Pro Second Speaker",
    "Con Second Speaker",
]

MODEL_POOL = ["GPT-5", "Claude", "DeepSeek", "Baidu"]

def role_permutation(seed: Optional[int] = None) -> Dict[str, str]:
    rng = random.Random(seed)
    names = MODEL_POOL[:]
    rng.shuffle(names)
    return dict(zip(ROLES_ORDER, names))

def strategy_hint(round_name: str) -> str:
    # 简洁版本（不含 emoji；利于兼容）
    if "Opening" in round_name:
        return ("Opening strategy: state a clear claim, give your strongest reason "
                "or example, anticipate one counter-point.")
    if "Rebuttal" in round_name or "Cross" in round_name:
        return ("Rebuttal strategy: target a core point from the previous speaker "
                "with evidence; expose a contradiction or missing premise.")
    if "Closing" in round_name:
        return ("Closing strategy: synthesize the debate, highlight unanswered issues, "
                "acknowledge trade-offs, end with a decisive takeaway.")
    return ("General strategy: respond concisely and advance your side with logic.")

def build_prompt(topic: str, role: str, round_name: str, previous: str = "") -> str:
    rules = (
        "Output rules:\n"
        "- Plain text only. No markdown or special formatting.\n"
        "- No greetings, no role/topic restatement.\n"
        "- Opening/Rebuttal/Cross-examination: exactly 5 sentences (<=50 words each).\n"
        "- Closing: exactly 4 sentences (<=50 words each).\n"
        "Content:\n"
        "1) Start with a clear claim.\n"
        "2) Give your strongest reason or concrete example (no fabricated stats).\n"
        "3) Directly address one key point from the previous speaker (quote <=10 words), then rebut.\n"
        "4) Weigh risks/tradeoffs and explain why your side minimizes the worst plausible outcome.\n"
        "5) Final sentence: either a sharp question or decisive takeaway.\n"
    )
    pos = "Support the proposition" if "Pro" in role else "Oppose the proposition"
    prev = f"\nPrevious speaker said:\n{previous}\n" if previous else ""
    return (
        f"You are participating in a debate about: {topic}\n"
        f"Your role: {role}. {pos}.\n"
        f"Current round: {round_name}\n"
        f"{prev}\n"
        f"{rules}\n"
        f"Strategy: {strategy_hint(round_name)}\n"
        f"Please deliver your statement now."
    )

def call_model(model_name: str, prompt: str) -> str:
    if model_name == "GPT-5":
        return call_openai_gpt5(prompt)
    if model_name == "Claude":
        return call_anthropic_claude(prompt)
    if model_name == "DeepSeek":
        return call_deepseek(prompt)
    if model_name == "Baidu":
        return call_baidu_ernie(prompt)
    return "[Unknown model]"

@dataclass
class SessionResult:
    debate_id: str
    roles: Dict[str, str]
    records: List[Dict]

def run_one_session(topic_text: str, topic_number: str, lang: str, debug: bool=False) -> SessionResult:
    """
    Run a single-language debate session (8 speeches).
    Context isolation = 每个 session 单独运行，不共享任何状态。
    """
    debate_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    roles = role_permutation()  # 独立均匀置换
    if debug:
        print(f"[Session start] debate_id={debate_id}, lang={lang}, roles={roles}")

    history: List[Tuple[str, str]] = []  # (speaker, text)
    detailed: List[Dict] = []

    for i, (role, round_name) in enumerate(DEBATE_ROUNDS):
        model_name = roles[role]
        prev_text = history[-1][1] if i > 0 else ""
        prompt = build_prompt(topic_text, role, round_name, previous=prev_text)
        t0 = time.time()
        text = call_model(model_name, prompt).strip()
        dt = time.time() - t0

        detailed.append({
            "debate_id": debate_id,
            "topic_number": topic_number,
            "topic": topic_text,
            "round_number": i + 1,
            "round_name": round_name,
            "speaker_role": role,
            "ai_model": model_name,
            "position": "Pro" if "Pro" in role else "Con",
            "final_speech": text,
            "speech_length": len(text),
            "timestamp": datetime.now().isoformat(),
            "previous_speaker": history[-1][0] if i > 0 else "",
            "response_time_seconds": round(dt, 2),
        })
        history.append((f"{model_name}({role})", text))

        if debug:
            print(f"  - {round_name} | {model_name} -> {len(text)} chars in {dt:.2f}s")

        if not DRY_RUN:
            time.sleep(random.uniform(1.1, 2.0))  # 轻微限速

    return SessionResult(debate_id=debate_id, roles=roles, records=detailed)


# ==================== Text Export ====================

def render_transcript(records: List[Dict]) -> str:
    lines = []
    for r in records:
        lines.append(f"[{r['round_name']}] {r['ai_model']} ({r['speaker_role']}): {r['final_speech']}")
    return "\n".join(lines)

def write_index_row(index_csv: str, row: Dict):
    write_header = not os.path.exists(index_csv)
    with open(index_csv, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=[
            "draw_id", "debate_id", "domain", "order", "session_lang",
            "roles", "models", "motion_en", "motion_zh", "outfile", "timestamp"
        ])
        if write_header:
            w.writeheader()
        w.writerow(row)


# ==================== Pilot Runner ====================

def run_pilot(csv_path: str, outdir: str, n_draws: int, min_domains: int, seed: int, debug: bool):
    random.seed(seed)
    ensure_outdir(outdir)

    df = pd.read_csv(csv_path)
    if "Motion" not in df.columns or "Motion_Chinese" not in df.columns:
        raise ValueError("CSV must contain columns: 'Motion' and 'Motion_Chinese'.")

    domain_col = choose_domain_column(df)
    draws = stratified_sample(df, domain_col, n_draws=n_draws, min_domains=min_domains, seed=seed)
    orders = make_order_vector(len(draws), seed=seed)

    index_csv = os.path.join(outdir, "sessions_index.csv")
    zh_first, en_first = 0, 0

    for i, row in draws.reset_index(drop=True).iterrows():
        motion_en = str(row["Motion"])
        motion_zh = str(row["Motion_Chinese"])
        domain = str(row[domain_col]) if domain_col else "NA"
        draw_id = f"{i+1:03d}"

        order = orders[i]
        if order == "zh-first":
            sequence = [("zh", motion_zh), ("en", motion_en)]
            zh_first += 1
        else:
            sequence = [("en", motion_en), ("zh", motion_zh)]
            en_first += 1

        domain_slug = slugify(domain) if domain != "NA" else "NA"
        order_tag = "zhfirst" if order == "zh-first" else "enfirst"

        for lang, topic_text in sequence:
            topic_number = f"{draw_id}_{lang}"
            sess = run_one_session(topic_text=topic_text, topic_number=topic_number, lang=lang, debug=debug)

            transcript = render_transcript(sess.records)
            outfile = os.path.join(outdir, f"s{draw_id}_{lang}_{domain_slug}_{order_tag}_{sess.debate_id}.txt")
            with open(outfile, "w", encoding="utf-8") as f:
                f.write(transcript)

            write_index_row(index_csv, {
                "draw_id": draw_id,
                "debate_id": sess.debate_id,
                "domain": domain,
                "order": order,
                "session_lang": lang,
                "roles": json.dumps(sess.roles, ensure_ascii=False),
                "models": ",".join(sess.roles.values()),
                "motion_en": motion_en,
                "motion_zh": motion_zh,
                "outfile": outfile,
                "timestamp": datetime.now().isoformat(),
            })

            if debug:
                print(f"[Saved] {outfile}")

            if not DRY_RUN:
                time.sleep(random.uniform(0.8, 1.6))

    print("\n=== Pilot Summary ===")
    print(f"- Draws (pairs): {len(draws)}")
    print(f"- Sessions saved: {len(draws) * 2} -> in {outdir}/")
    if domain_col:
        vc = draws[domain_col].value_counts()
        print(f"- Domain coverage: {len(vc)}")
        print(vc.to_string())
    print(f"- Order split: zh-first={zh_first}, en-first={en_first}")
    print(f"- Index file : {index_csv}")
    if DRY_RUN:
        print("[NOTE] DRY_RUN=1 (API calls skipped; placeholder text written.)")


# ==================== CLI ====================
# 在Jupyter中测试时使用
if 'ipykernel_launcher' in sys.argv[0]:
    import sys
    sys.argv = [
        'script.py',
        '--csv', 'debate_topic.csv',  # 修改为您的CSV文件路径
        '--outdir', 'sessions',
        '--n_draws', '1',
        '--min_domains', '1',
        '--seed', '42',
        '--debug', '1'
    ]

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--csv", required=True, help="Path to debate_topic.csv (must contain Motion, Motion_Chinese)")
    parser.add_argument("--outdir", default="sessions", help="Directory to save session .txt files")
    parser.add_argument("--n_draws", type=int, default=1, help="Number of motion pairs to run (10–15 recommended)")
    parser.add_argument("--min_domains", type=int, default=4, help="Minimal distinct domains to cover (3–5)")
    parser.add_argument("--seed", type=int, default=42, help="Random seed")
    parser.add_argument("--debug", type=int, default=0, help="Debug prints (1=yes)")
    args = parser.parse_args()

    run_pilot(
        csv_path=args.csv,
        outdir=args.outdir,
        n_draws=args.n_draws,
        min_domains=args.min_domains,
        seed=args.seed,
        debug=bool(args.debug),
    )


[WARN] No domain column; using simple random sample.
[Session start] debate_id=20251108_143433, lang=en, roles={'Pro First Speaker': 'GPT-5', 'Con First Speaker': 'DeepSeek', 'Pro Second Speaker': 'Claude', 'Con Second Speaker': 'Baidu'}
  - Round 1: Pro Opening Statement | GPT-5 -> 1106 chars in 37.20s
  - Round 1: Con Opening Statement | DeepSeek -> 174 chars in 0.33s
  - Round 2: Pro Rebuttal/Supplement | Claude -> 192 chars in 0.13s


KeyboardInterrupt: 